In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage.interpolation import map_coordinates
from skimage.transform import resize

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Project3 

/content/drive/MyDrive/Project3


Loading data

In [ ]:

X_train = torch.load('datasets/X_train')
X_test = torch.load('datasets/X_test')
y_train = torch.load('datasets/y_train')
y_test = torch.load('datasets/y_test')

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
from torchsummary import summary

In [ ]:
y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)

In [ ]:
def convert(X,a,b):
    X = X.reshape(X.shape[0],1,a,a)
    # X = resize(X, (X.shape[0],1,a+1,b))
    X = X.astype(np.float32)
    X = torch.tensor(X)
    return X

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [ ]:
def convertDataloader(X1,y_train):
    batch_size = 64
    train_data = TensorDataset(X1,y_train)
    train_sampler = SequentialSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return train_dataloader

In [ ]:
def setDataloader(X_train,X_test,y_train,y_test):
    X1 = convert(X_train,28,28)
    X1_test = convert(X_test,28,28)
    train_dataloader = convertDataloader(X1,y_train)
    test_dataloader = convertDataloader(X1_test,y_test)
    return train_dataloader,test_dataloader

Convert dataloader with a batch: 64

In [ ]:
train_dataloader,test_dataloader = setDataloader(X_train,X_test,y_train,y_test)

Training model

In [ ]:
from tqdm import tqdm_notebook


# Keep track of losses for plotting


from sklearn.metrics import accuracy_score
def trainEpoch(net,criterion,net_optimizer):
  
    epochs = 50
    
    bestScore = 0.993

    for epoch in range(epochs):
        print('-'*40)
        print('Epoch:',epoch)
        net.train()
        y_pred = []
        y_true = []
        train_dataloader,test_dataloader = setDataloader(X_train,X_test,y_train,y_test)

        for i, batch in tqdm_notebook(enumerate(train_dataloader)):
            net.zero_grad()

            input1 = batch[0].to(device)
            

            y = batch[1].to(device)
            output = net(input1)
            loss = criterion(output, y)
            loss.backward()
            net_optimizer.step()

            y_direc = torch.argmax(output,dim = 1)
            y_pred+= y_direc.cpu()
            y_true+=y.cpu()

        
        y_pred = np.array(y_pred)
        y_true = np.array(y_true)
        print("Train Accuracy",accuracy_score(y_true, y_pred))

        net.eval()
        with torch.no_grad():
            y_pred = []
            y_true = []
            for i, batch in tqdm_notebook(enumerate(test_dataloader)):
                net.zero_grad()

                input1 = batch[0].to(device)
                
                y = batch[1].to(device)
                output = net(input1)
                
                y_direc = torch.argmax(output,dim = 1)
                y_pred+= y_direc.cpu()
                y_true+=y.cpu()
            score = accuracy_score(y_true, y_pred)
            print("Test Accuracy",score)
            # if(score > bestScore):
            #     bestScore = score
            #     torch.save({
            #       'epoch': epoch,
            #       'model_state_dict': net.state_dict(),
            #       'optimizer_state_dict': net_optimizer.state_dict(),
            #       'best_score': bestScore
            #       }, 'weights/model.pth')



      
    return 0



In [ ]:
from torch import optim

Architecture of model

In [ ]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.pool1 = nn.MaxPool2d(kernel_size=3,stride=1)

        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.pool2 = nn.MaxPool2d(kernel_size=3,stride=1)

        self.conv3 = nn.Conv2d(64, 128, 5, 1)
        self.pool3 = nn.MaxPool2d(kernel_size=3,stride=1)

        self.conv4 = nn.Conv2d(128, 256, 5, 1)
        self.pool4 = nn.MaxPool2d(kernel_size=3,stride=1)

        self.conv5 = nn.Conv2d(256, 256, 5, 1)
        self.pool5 = nn.MaxPool2d(kernel_size=2,stride=2)


        self.dropout1 = nn.Dropout(0.1)


        self.fc1 = nn.Linear(1024, 128)
        self.fc2 = nn.Linear(128, 10)
        self.logSoftmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x1 = self.conv1(x)
        x1 = nn.functional.normalize(x1,dim=3)
        x1 = self.pool1(F.relu(x1))
        x1 = self.dropout1(x1)

        x2 = self.conv2(x1)
        x2 = nn.functional.normalize(x2,dim=3)
        x2 = self.pool2(F.relu(x2))
        x2 = self.dropout1(x2)

        x3 = self.conv3(x2)
        x3 = nn.functional.normalize(x3,dim=3)
        x3 = self.pool3(F.relu(x3))
        x3 = self.dropout1(x3)

        x4 = self.conv4(x3)
        x4 = nn.functional.normalize(x4,dim=3)
        x4 = self.pool4(F.relu(x4))
        x4 = self.dropout1(x4)

        x5 = self.conv5(x4)
        x5 = nn.functional.normalize(x5,dim=3)
        x5 = self.pool5(F.relu(x5))
        x5 = self.dropout1(x5)

        x6 = torch.flatten(x5, 1)
        # print(x6.shape)

        out = self.fc1(x6)
        out = F.relu(out)
        out = self.dropout1(out)
        
        out = self.fc2(out)
        output = self.logSoftmax(out)
        return output

Initialize parameter: optim, criterion

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
net = Net1().to(device)
net_optimizer = optim.Adam(lr=0.001,params= net.parameters())

In [ ]:
summary(net,(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
         MaxPool2d-2           [-1, 32, 24, 24]               0
           Dropout-3           [-1, 32, 24, 24]               0
            Conv2d-4           [-1, 64, 22, 22]          18,496
         MaxPool2d-5           [-1, 64, 20, 20]               0
           Dropout-6           [-1, 64, 20, 20]               0
            Conv2d-7          [-1, 128, 16, 16]         204,928
         MaxPool2d-8          [-1, 128, 14, 14]               0
           Dropout-9          [-1, 128, 14, 14]               0
           Conv2d-10          [-1, 256, 10, 10]         819,456
        MaxPool2d-11            [-1, 256, 8, 8]               0
          Dropout-12            [-1, 256, 8, 8]               0
           Conv2d-13            [-1, 256, 4, 4]       1,638,656
        MaxPool2d-14            [-1, 25

In [ ]:
trainEpoch(net,criterion,net_optimizer)

----------------------------------------
Epoch: 0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

Train Accuracy 0.9954


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

0it [00:00, ?it/s]

Train Accuracy 0.9969833333333333


0it [00:00, ?it/s]

Test Accuracy 0.9924


0

In [ ]:
block = torch.load('weights/model.pth')

In [ ]:
block['best_score']

0.9939